In [1]:
import json
import pandas as pd
from textblob import TextBlob

# Cargar el archivo JSON
with open( r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\Nuevo Analisis de Sentimientos\productos.json', encoding='utf-8') as file:
    data = json.load(file)

# Almacenar resultados
productos_sentimiento = []

# Procesar cada producto
for producto in data:
    title = producto['title']
    category = producto['category']
    reviews = producto['reviews']
    
    if reviews:  # Verificar si hay reseñas
        # Calcular el sentimiento para cada reseña
        sentimientos = [TextBlob(review).sentiment.polarity for review in reviews]
        # Calcular el sentimiento promedio del producto
        sentimiento_promedio = sum(sentimientos) / len(sentimientos)
    else:
        # Si no hay reseñas, asignar un valor predeterminado
        sentimientos = []
        sentimiento_promedio = 0  # Puedes usar "None" o "No aplica" si prefieres
    
    # Almacenar resultados del producto
    productos_sentimiento.append({
        'Producto': title,
        'Categoría': category,
        'Sentimiento promedio': sentimiento_promedio,
        'Sentimientos individuales': sentimientos
    })

# Crear un DataFrame
df_productos = pd.DataFrame(productos_sentimiento)

# Mostrar el resultado
print(df_productos)

# Guardar los resultados en un archivo JSON
output_json_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\Nuevo Analisis de Sentimientos\productos_sentimiento.json'
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(productos_sentimiento, json_file, ensure_ascii=False, indent=4)

# Guardar los resultados en un archivo CSV
output_csv_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\Nuevo Analisis de Sentimientos\productos_sentimiento.csv'
df_productos.to_csv(output_csv_path, index=False, encoding='utf-8')


                                              Producto    Categoría  \
0    HP 14 inch Laptop, HD Display, 12th Generation...      laptops   
1    15.6" FHD IPS Business Laptop, 32GB RAM 1TB SS...      laptops   
2    ASUS 2024 Vivobook Go 11.6” HD Laptop, Intel D...      laptops   
3    ASUS 15.6” Vivobook Go Laptop, Intel Dual Core...      laptops   
4    HP 17 Laptop, 17.3" HD+ Touchscreen Display, 1...      laptops   
..                                                 ...          ...   
353  32GB FRAMEO 10.1 Inch Smart WiFi Digital Photo...  electronics   
354  Mini Projector, VISSPL Full HD 1080P Video Pro...  electronics   
355  Blink Outdoor 4 [Newest Model] - Wireless outd...  electronics   
356  Roku Express 4K+ | Roku Streaming Device 4K/HD...  electronics   
357  Skylight Calendar: 15 inch Digital Calendar & ...  electronics   

     Sentimiento promedio                          Sentimientos individuales  
0                0.087051  [0.2357142857142857, -0.07222222222222224

In [3]:
# Agrupar por categoría y calcular métricas
df_categorias = df_productos.groupby('Categoría').agg(
    Sentimiento_promedio=('Sentimiento promedio', 'mean'),
    Productos_positivos=('Sentimiento promedio', lambda x: (x > 0).sum()),
    Productos_negativos=('Sentimiento promedio', lambda x: (x < 0).sum()),
    Productos_neutros=('Sentimiento promedio', lambda x: (x == 0).sum())
).reset_index()

# Mostrar resultados por categoría
print(df_categorias)

# Guardar el DataFrame agrupado en un archivo CSV
output_categorias_csv_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\categorias_sentimiento.csv'
df_categorias.to_csv(output_categorias_csv_path, index=False, encoding='utf-8')

# Guardar el DataFrame agrupado en un archivo JSON
output_categorias_json_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\categorias_sentimiento.json'
df_categorias.to_json(output_categorias_json_path, orient='records', force_ascii=False, indent=4)

print(f"Archivo CSV guardado en: {output_categorias_csv_path}")
print(f"Archivo JSON guardado en: {output_categorias_json_path}")



            Categoría  Sentimiento_promedio  Productos_positivos  \
0         electronics              0.120902                   16   
1  gaming_accessories              0.253154                  190   
2             laptops              0.291888                   63   
3      pc_peripherals              0.191405                   62   

   Productos_negativos  Productos_neutros  
0                    0                 16  
1                    1                  1  
2                    2                  5  
3                    0                  2  
Archivo CSV guardado en: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\categorias_sentimiento.csv
Archivo JSON guardado en: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\categorias_sentimiento.json


In [7]:
# Identificar los productos con mejor, peor y nulos sentimientos promedio
productos_mejor_sentimiento = df_productos[df_productos['Sentimiento promedio'] >= 0.5]
productos_peor_sentimiento = df_productos[df_productos['Sentimiento promedio'] < 0]
productos_sentimiento_nulo = df_productos[df_productos['Sentimientos individuales'].apply(len) == 0]

# Guardar en CSV y JSON
base_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos'

# Guardar los productos con mejor sentimiento promedio
mejor_csv_path = f'{base_path}\productos_mejor_sentimiento.csv'
mejor_json_path = f'{base_path}\productos_mejor_sentimiento.json'
productos_mejor_sentimiento.to_csv(mejor_csv_path, index=False, encoding='utf-8')
productos_mejor_sentimiento.to_json(mejor_json_path, orient='records', force_ascii=False, indent=4)

# Guardar los productos con peor sentimiento promedio
peor_csv_path = f'{base_path}\productos_peor_sentimiento.csv'
peor_json_path = f'{base_path}\productos_peor_sentimiento.json'
productos_peor_sentimiento.to_csv(peor_csv_path, index=False, encoding='utf-8')
productos_peor_sentimiento.to_json(peor_json_path, orient='records', force_ascii=False, indent=4)

# Guardar los productos con sentimiento nulo
nulo_csv_path = f'{base_path}\productos_sentimiento_nulo.csv'
nulo_json_path = f'{base_path}\productos_sentimiento_nulo.json'
productos_sentimiento_nulo.to_csv(nulo_csv_path, index=False, encoding='utf-8')
productos_sentimiento_nulo.to_json(nulo_json_path, orient='records', force_ascii=False, indent=4)

# Imprimir mensajes de confirmación
print(f"Productos con mejor sentimiento guardados en:\n  CSV: {mejor_csv_path}\n  JSON: {mejor_json_path}")
print(f"Productos con peor sentimiento guardados en:\n  CSV: {peor_csv_path}\n  JSON: {peor_json_path}")
print(f"Productos con sentimiento nulo guardados en:\n  CSV: {nulo_csv_path}\n  JSON: {nulo_json_path}")


Productos con mejor sentimiento guardados en:
  CSV: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_mejor_sentimiento.csv
  JSON: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_mejor_sentimiento.json
Productos con peor sentimiento guardados en:
  CSV: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_peor_sentimiento.csv
  JSON: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_peor_sentimiento.json
Productos con sentimiento nulo guardados en:
  CSV: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_sentimiento_nulo.csv
  JSON: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_sentimiento_nulo.json


In [8]:
# Definir base_path correctamente con barras dobles
base_path = r'C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos'

# Guardar productos sin reseñas
sin_reseñas_csv_path = f'{base_path}\\productos_sin_reseñas.csv'
sin_reseñas_json_path = f'{base_path}\\productos_sin_reseñas.json'
productos_sin_reseñas.to_csv(sin_reseñas_csv_path, index=False, encoding='utf-8')
productos_sin_reseñas.to_json(sin_reseñas_json_path, orient='records', force_ascii=False, indent=4)

# Guardar los 20 mejores productos
mejor_csv_path = f'{base_path}\\top_20_mejor_sentimiento.csv'
mejor_json_path = f'{base_path}\\top_20_mejor_sentimiento.json'
productos_mejor_sentimiento.to_csv(mejor_csv_path, index=False, encoding='utf-8')
productos_mejor_sentimiento.to_json(mejor_json_path, orient='records', force_ascii=False, indent=4)

# Guardar los 20 peores productos
peor_csv_path = f'{base_path}\\top_20_peor_sentimiento.csv'
peor_json_path = f'{base_path}\\top_20_peor_sentimiento.json'
productos_peor_sentimiento.to_csv(peor_csv_path, index=False, encoding='utf-8')
productos_peor_sentimiento.to_json(peor_json_path, orient='records', force_ascii=False, indent=4)

# Imprimir mensajes de confirmación
print(f"Productos sin reseñas guardados en:\n  CSV: {sin_reseñas_csv_path}\n  JSON: {sin_reseñas_json_path}")
print(f"Top 20 mejores productos guardados en:\n  CSV: {mejor_csv_path}\n  JSON: {mejor_json_path}")
print(f"Top 20 peores productos guardados en:\n  CSV: {peor_csv_path}\n  JSON: {peor_json_path}")


Productos sin reseñas guardados en:
  CSV: C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos\productos_sin_reseñas.csv
  JSON: C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos\productos_sin_reseñas.json
Top 20 mejores productos guardados en:
  CSV: C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos\top_20_mejor_sentimiento.csv
  JSON: C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos\top_20_mejor_sentimiento.json
Top 20 peores productos guardados en:
  CSV: C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos\top_20_peor_sentimiento.csv
  JSON: C:\\Users\\JP\\Desktop\\Archivos para Analisis de Sentimientos\top_20_peor_sentimiento.json


In [9]:
import os

# Definir base_path
base_path = r'C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos'

# Agregar una columna de ID única
df_productos['ID'] = range(1, len(df_productos) + 1)

# Construir rutas de archivo
productos_con_id_csv_path = os.path.join(base_path, 'productos_con_id.csv')
productos_con_id_json_path = os.path.join(base_path, 'productos_con_id.json')

# Guardar el DataFrame con IDs únicos en CSV y JSON
df_productos.to_csv(productos_con_id_csv_path, index=False, encoding='utf-8')
df_productos.to_json(productos_con_id_json_path, orient='records', force_ascii=False, indent=4)

# Mensajes de confirmación
print(f"Productos con IDs únicos guardados en:\n  CSV: {productos_con_id_csv_path}\n  JSON: {productos_con_id_json_path}")



Productos con IDs únicos guardados en:
  CSV: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_con_id.csv
  JSON: C:\Users\JP\Desktop\Archivos para Analisis de Sentimientos\productos_con_id.json
